This is the aircraft gate assignment problem for the Operations optimization course. 

In [252]:
# Gurobi for optimization
from gurobipy import GRB, Model, quicksum
import gurobipy as gp

# Pandas for data manipulation
import pandas as pd

# Numpy for numerical operations
import numpy as np

# Matplotlib for visualization
import matplotlib.pyplot as plt

# Datetime for handling time-related data
import datetime

# Pickle for saving/loading objects (optional, based on need)
import pickle



In [253]:
# Sample Data Generation
num_flights = 10
num_gates = 6

# Seed for reproducibility
np.random.seed(46)

# Generate flight data
flight_ids = [f"F{i+1}" for i in range(num_flights)]
arrival_times = np.random.randint(0, 700, num_flights)  # minutes from midnight
departure_times = arrival_times + np.random.randint(60, 180, num_flights)  # ensure at least 1 hour turnaround
passenger_numbers = np.random.randint(50, 300, num_flights)
flight_types = np.random.choice(['domestic', 'international'], num_flights)

flight_data = pd.DataFrame({
    'flight_id': flight_ids,
    'arrival_time': arrival_times,
    'departure_time': departure_times,
    'passenger_number': passenger_numbers,
    'flight_type': flight_types
})

# Generate gate data
gate_ids = [f"G{i+1}" for i in range(num_gates)]
gate_types = np.random.choice(['domestic', 'international'], num_gates)

gate_data = pd.DataFrame({
    'gate_id': gate_ids,
    'gate_type': gate_types
})

# Calculate distances between gates (d_kl) and from gates to entrance/exit (ed_k)
distances = np.random.randint(100, 1000, (num_gates, num_gates))
np.fill_diagonal(distances, 0)
distance_df = pd.DataFrame(distances, index=gate_ids, columns=gate_ids)
ed_k = np.random.randint(100, 500, num_gates)

# Generate transiting passengers data (p_ij) with constraints
p_ij = np.zeros((num_flights, num_flights), dtype=int)

for i in range(num_flights):
    max_transit_passengers = passenger_numbers[i] // 2  # Maximum of 50% of the total passengers
    remaining_passengers = max_transit_passengers
    for j in np.random.permutation(num_flights):  # Randomly permute to distribute passengers
        if i != j and remaining_passengers > 0:
            p_ij[i, j] = np.random.randint(0, min(50, remaining_passengers + 1))
            remaining_passengers -= p_ij[i, j]

# Verify generated data
print("Flight Data:")
print(flight_data.head(num_flights))

print("\nGate Data:")
print(gate_data.head(num_gates))

print("\nDistance Matrix (d_kl):")
print(distance_df)

print("\nDistance to Entrance/Exit (ed_k):")
print(pd.Series(ed_k, index=gate_ids))

print("\nTransiting Passengers (p_ij):")
print(pd.DataFrame(p_ij, index=flight_ids, columns=flight_ids).head())


Flight Data:
  flight_id  arrival_time  departure_time  passenger_number    flight_type
0        F1           189             281               164       domestic
1        F2           552             643               200       domestic
2        F3           372             487               265       domestic
3        F4           442             524               250       domestic
4        F5           403             561               170  international
5        F6           658             777               168       domestic
6        F7           316             455               226  international
7        F8           411             501                52       domestic
8        F9           642             800               285       domestic
9       F10            72             134               157       domestic

Gate Data:
  gate_id      gate_type
0      G1  international
1      G2  international
2      G3  international
3      G4  international
4      G5       domestic


### Model Explanation

The objective of the FC formulation is to minimize the total walking distance of passengers at the airport, considering both transfer passengers and non-transfer passengers. The model includes the following key components:

- **Parameters**:
  - `num_flights`: Number of flights
  - `num_gates`: Number of gates
  - `arrival_times`: Arrival times of flights
  - `departure_times`: Departure times of flights
  - `passenger_numbers`: Number of passengers per flight
  - `flight_types`: Type of flight (domestic/international)
  - `gate_types`: Type of gate (domestic/international)
  - `distances`: Distance matrix between gates

- **Decision Variables**:
  - `x[i, k]`: Binary variable, 1 if flight \(i\) is assigned to gate \(k\), 0 otherwise
  - `w[i, k, l]`: Continuous variable representing the number of passengers from flight \(i\) through gate \(k\) to gate \(l\)

- **Objective Function**:
  - Minimize the total walking distance of passengers.

- **Constraints**:
  - Each flight must be assigned to exactly one gate.
  - Gates cannot be double-booked.
  - Flow balance constraints for transfer passengers.


In [254]:
# Initialize the Gurobi model
model = gp.Model("FC_Formulation")

# Assign parameters from simulated data
num_flights = len(flight_data)
num_gates = len(gate_data)

# Distance matrices
d_kl = np.zeros((num_gates + 1, num_gates + 1))
d_kl[:num_gates, :num_gates] = distance_df.values
ed_k = np.append(ed_k, 0)  # Append zero for the apron distance

# Passengers data
e_i = flight_data['passenger_number'].values - np.sum(p_ij, axis=1)
f_i = flight_data['passenger_number'].values - np.sum(p_ij, axis=0)

# Aircraft type (domestic/international)
g_i = flight_data['flight_type'].apply(lambda x: 'D' if x == 'domestic' else 'I').values

# Sets of domestic and international gates
K_D = [k for k, t in zip(range(num_gates), gate_types) if t == 'domestic']
K_I = [k for k, t in zip(range(num_gates), gate_types) if t == 'international']
K_D.append(num_gates)  # Add apron to domestic gates
K_I.append(num_gates)  # Add apron to international gates

# Number of gates
m = num_gates

# Generate overlap sets
def generate_overlap_sets(flight_data):
    I_Dt = []
    I_It = []
    T_D = {}
    T_I = {}
    
    domestic_flights = flight_data[flight_data['flight_type'] == 'domestic']
    international_flights = flight_data[flight_data['flight_type'] == 'international']
    
    # Generate overlap sets for domestic flights
    for i in range(len(domestic_flights)):
        overlap_set = set()
        for j in range(len(domestic_flights)):
            if i != j and not (domestic_flights.iloc[i]['departure_time'] <= domestic_flights.iloc[j]['arrival_time'] or
                               domestic_flights.iloc[j]['departure_time'] <= domestic_flights.iloc[i]['arrival_time']):
                overlap_set.add(domestic_flights.iloc[j]['flight_id'])
                overlap_set.add(domestic_flights.iloc[i]['flight_id'])
        if overlap_set:
            I_Dt.append(overlap_set)
    
    # Generate overlap sets for international flights
    for i in range(len(international_flights)):
        overlap_set = set()
        for j in range(len(international_flights)):
            if i != j and not (international_flights.iloc[i]['departure_time'] <= international_flights.iloc[j]['arrival_time'] or
                               international_flights.iloc[j]['departure_time'] <= international_flights.iloc[i]['arrival_time']):
                overlap_set.add(international_flights.iloc[j]['flight_id'])
                overlap_set.add(international_flights.iloc[i]['flight_id'])
        if overlap_set:
            I_It.append(overlap_set)
    
    # Generate T_D and T_I
    for idx, overlap_set in enumerate(I_Dt):
        if overlap_set:
            T_D[f"TD{idx+1}"] = overlap_set
            
    for idx, overlap_set in enumerate(I_It):
        if overlap_set:
            T_I[f"TI{idx+1}"] = overlap_set
    
    return T_D, T_I

T_D, T_I = generate_overlap_sets(flight_data)

# Print overlap sets
print("Overlap Sets for Domestic Flights (T_D):")
for key, value in T_D.items():
    print(f"{key}: {value}")

print("\nOverlap Sets for International Flights (T_I):")
for key, value in T_I.items():
    print(f"{key}: {value}")

# Calculate N_A_star
def calculate_N_A_star(T_D, T_I, K_D, K_I):
    N_A_star = 0
    for overlap_set in T_D.values():
        if len(overlap_set) > len(K_D):
            N_A_star += len(overlap_set) - len(K_D)
            print(f"Domestic Overlap Set: {overlap_set}, Excess: {len(overlap_set) - len(K_D)}")
    for overlap_set in T_I.values():
        if len(overlap_set) > len(K_I):
            N_A_star += len(overlap_set) - len(K_I)
            print(f"International Overlap Set: {overlap_set}, Excess: {len(overlap_set) - len(K_I)}")
    return N_A_star

N_A_star = calculate_N_A_star(T_D, T_I, K_D, K_I)
print("\nN_A_star:", N_A_star)

# Decision Variables
x = model.addVars(num_flights, num_gates + 1, vtype=GRB.BINARY, name="x")
w = model.addVars(num_flights, num_gates + 1, num_gates + 1, vtype=GRB.CONTINUOUS, name="w")

# Verify variables
print("Decision variables defined.")


Overlap Sets for Domestic Flights (T_D):
TD1: {'F9', 'F2'}
TD2: {'F4', 'F8', 'F3'}
TD3: {'F4', 'F8', 'F3'}
TD4: {'F9', 'F6'}
TD5: {'F4', 'F8', 'F3'}
TD6: {'F9', 'F2', 'F6'}

Overlap Sets for International Flights (T_I):
TI1: {'F5', 'F7'}
TI2: {'F5', 'F7'}
Domestic Overlap Set: {'F4', 'F8', 'F3'}, Excess: 1
Domestic Overlap Set: {'F4', 'F8', 'F3'}, Excess: 1
Domestic Overlap Set: {'F4', 'F8', 'F3'}, Excess: 1
Domestic Overlap Set: {'F9', 'F2', 'F6'}, Excess: 1

N_A_star: 4
Decision variables defined.


In [255]:
# Objective function components\n",
objective = quicksum(d_kl[k, l] * w[i, k, l] for i in range(num_flights) for k in range(num_gates + 1) for l in range(num_gates + 1)) + \
            quicksum((e_i[i] + f_i[i]) * ed_k[k] * x[i, k] for i in range(num_flights) for k in range(num_gates + 1))

model.setObjective(objective, GRB.MINIMIZE)

# Verify objective
print("Objective function defined.")

Objective function defined.


In [256]:
# Constraint 3: Assign each aircraft to exactly one gate
for i in range(num_flights):
    model.addConstr(quicksum(x[i, k] for k in range(num_gates + 1)) == 1, name=f"assign_one_gate_{i}")

# Constraint 4: No overlapping aircraft at the same gate for domestic flights
for key, overlap_set in T_D.items():
    for gate in K_D:
        if gate != num_gates:  # Exclude the apron
            model.addConstr(quicksum(x[flight_ids.index(flight), gate] for flight in overlap_set) <= 1, name=f"overlap_dom_{key}_{gate}")

# Constraint 5: No overlapping aircraft at the same gate for international flights
for key, overlap_set in T_I.items():
    for gate in K_I:
        if gate != num_gates:  # Exclude the apron
            model.addConstr(quicksum(x[flight_ids.index(flight), gate] for flight in overlap_set) <= 1, name=f"overlap_int_{key}_{gate}")

# Constraint 6: Minimum apron assignments
model.addConstr(quicksum(x[i, num_gates] for i in range(num_flights)) == N_A_star, name="min_apron_assignments")

# Constraint 10: Passenger flow balance constraints (outbound)
for i in range(num_flights):
    for k in range(num_gates + 1):
        if k in (K_D if g_i[i] == 'D' else K_I):
            model.addConstr(
                quicksum(w[i, k, l] for l in range(num_gates + 1)) == x[i, k] * sum(p_ij[i, j] for j in range(num_flights)),
                name=f"flow_balance_out_{i}_{k}"
            )

# Constraint 11: Passenger flow balance constraints (inbound)
for i in range(num_flights):
    for k in range(num_gates + 1):
        model.addConstr(
            quicksum(w[i, l, k] for l in range(num_gates + 1)) == sum(p_ij[j, i] * x[j, k] for j in range(num_flights)),
            name=f"flow_balance_in_{i}_{k}"
        )

# Constraint 12: Non-negativity of flow variables
for i in range(num_flights):
    for k in range(num_gates + 1):
        for l in range(num_gates + 1):
            if k in (K_D if g_i[i] == 'D' else K_I):
                model.addConstr(w[i, k, l] >= 0, name=f"nonneg_flow_{i}_{k}_{l}")

# # Constraint 13: Minimum apron assignments for overlapping domestic flights
# for key, overlap_set in T_D.items():
#     if len(overlap_set) > len(K_D) - 1:  # Exclude the apron when comparing gate counts
#         model.addConstr(
#             quicksum(x[flight_ids.index(flight), num_gates] for flight in overlap_set) >= len(overlap_set) - (len(K_D) - 1),
#             name=f"min_apron_dom_{key}"
#         )

# # Constraint 14: Minimum apron assignments for overlapping international flights
# for key, overlap_set in T_I.items():
#     if len(overlap_set) > len(K_I) - 1:  # Exclude the apron when comparing gate counts
#         model.addConstr(
#             quicksum(x[flight_ids.index(flight), num_gates] for flight in overlap_set) >= len(overlap_set) - (len(K_I) - 1),
#             name=f"min_apron_int_{key}"
#         )

# print("Constraints 13 and 14 defined.")

# # Constraint 3: Assign each aircraft to exactly one gate
# for i in range(num_flights):
#     model.addConstr(quicksum(x[i, k] for k in range(num_gates + 1)) == 1, name=f"assign_one_gate_{i}")

# # Constraint 4: No overlapping aircraft at the same gate for domestic flights
# for key, overlap_set in T_D.items():
#     for gate in K_D:
#         if gate != num_gates:  # Exclude the apron
#             model.addConstr(quicksum(x[flight_ids.index(flight), gate] for flight in overlap_set) <= 1, name=f"overlap_dom_{key}_{gate}")

# # Constraint 5: No overlapping aircraft at the same gate for international flights
# for key, overlap_set in T_I.items():
#     for gate in K_I:
#         if gate != num_gates:  # Exclude the apron
#             model.addConstr(quicksum(x[flight_ids.index(flight), gate] for flight in overlap_set) <= 1, name=f"overlap_int_{key}_{gate}")

# # Constraint 6: Minimum apron assignments
# model.addConstr(quicksum(x[i, num_gates] for i in range(num_flights)) >= N_A_star, name="min_apron_assignments")

# # Constraint 10: Passenger flow balance constraints (outbound)
# for i in range(num_flights):
#     for k in range(num_gates + 1):
#         model.addConstr(
#             quicksum(w[i, k, l] for l in range(num_gates + 1)) == x[i, k] * sum(p_ij[i, j] for j in range(num_flights)),
#             name=f"flow_balance_out_{i}_{k}"
#         )

# # Constraint 11: Passenger flow balance constraints (inbound)
# for i in range(num_flights):
#     for k in range(num_gates + 1):
#         model.addConstr(
#             quicksum(w[i, l, k] for l in range(num_gates + 1)) == sum(p_ij[j, i] * x[j, k] for j in range(num_flights)),
#             name=f"flow_balance_in_{i}_{k}"
#         )

# # Constraint 12: Non-negativity of flow variables
# for i in range(num_flights):
#     for k in range(num_gates + 1):
#         for l in range(num_gates + 1):
#             model.addConstr(w[i, k, l] >= 0, name=f"nonneg_flow_{i}_{k}_{l}")

# # Constraint 13: Minimum apron assignments for overlapping domestic flights
# for key, overlap_set in T_D.items():
#     if len(overlap_set) > len(K_D) - 1:  # Exclude the apron when comparing gate counts
#         model.addConstr(
#             quicksum(x[flight_ids.index(flight), num_gates] for flight in overlap_set) >= len(overlap_set) - (len(K_D) - 1),
#             name=f"min_apron_dom_{key}"
#         )

# # Constraint 14: Minimum apron assignments for overlapping international flights
# for key, overlap_set in T_I.items():
#     if len(overlap_set) > len(K_I) - 1:  # Exclude the apron when comparing gate counts
#         model.addConstr(
#             quicksum(x[flight_ids.index(flight), num_gates] for flight in overlap_set) >= len(overlap_set) - (len(K_I) - 1),
#             name=f"min_apron_int_{key}"
#         )




In [257]:
# Optimize the model
model.optimize()

# Check the optimization result
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    
    # assignments = model.getAttr('x', x)
    # for i in range(num_flights):
    #     assigned_gates = [k for k in range(num_gates + 1) if assignments[i, k] > 0.5]
    #     print(f"Flight {flight_ids[i]} assigned to Gates {assigned_gates}")

    # Retrieve the optimal assignments
    assignments = model.getAttr('x', x)
    flow_variables = model.getAttr('x', w)

    print("Gate Assignments:")
    for i in range(num_flights):
        for k in range(num_gates + 1):
            if assignments[i, k] > 0.5:
                print(f"Flight {flight_ids[i]} assigned to Gate {gate_ids[k] if k < num_gates else 'Apron'}")

    print("\nFlow Variables:")
    for i in range(num_flights):
        for k in range(num_gates + 1):
            for l in range(num_gates + 1):
                if flow_variables[i, k, l] > 0.5:
                    print(f"Flow from Flight {flight_ids[i]} at Gate {gate_ids[k] if k < num_gates else 'Apron'} to Gate {gate_ids[l] if l < num_gates else 'Apron'}: {flow_variables[i, k, l]}")
elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded")
else:
    print("Optimization ended with status", model.status)



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 321 rows, 560 columns and 1488 nonzeros
Model fingerprint: 0x78e5a437
Variable types: 490 continuous, 70 integer (70 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 205 rows and 281 columns
Presolve time: 0.00s
Presolved: 116 rows, 279 columns, 991 nonzeros
Variable types: 209 continuous, 70 integer (70 binary)

Root relaxation: objective 9.209520e+04, 55 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 92095.2000    0   18       